# Anime Recommendation System
We can create in different ways recommendation system
Today I will present an anime recommendation system using Content-based recommender systems

First, we import the necessary libraries

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np 
import pandas as pd 
import re

## 0. Data Collection 
To obtain the data, I used the previously written script in the web_scrapp.py file, which saves the data obtained from the myanimelist website in csv format

## 1. Data Preparation
To load and clean the data, we will use two python modules, i.e. pandas and regular expressions

In [2]:
data = pd.read_csv("anime_data.csv")
data.head()

,Title,Description,Type,Episodes,Source,Genres,Themes,Demographic,Rating
0,Shingeki no Kyojin,"Centuries ago, mankind was slaughtered to near...",TV,25,Manga,"'Action', 'Award Winning', 'Drama', 'Suspense'","'Gore', 'Military', 'Survival', 'Gore', 'Milit...",'Shounen',R - 17+ (violence & profanity)
1,Death Note,"Brutal murders, petty thefts, and senseless vi...",TV,37,Manga,"'Supernatural', 'Suspense'",'Psychological','Shounen',R - 17+ (violence & profanity)
2,Fullmetal Alchemist: Brotherhood,After a horrific alchemy experiment goes wrong...,TV,64,Manga,"'Action', 'Adventure', 'Drama', 'Fantasy'",'Military','Shounen',R - 17+ (violence & profanity)
3,One Punch Man,The seemingly unimpressive Saitama has a rathe...,TV,12,Web manga,"'Action', 'Comedy'","'Adult Cast', 'Parody', 'Super Power', 'Adult ...",'Seinen',R - 17+ (violence & profanity)
4,Sword Art Online,Ever since the release of the innovative Nerve...,TV,25,Light novel,"'Action', 'Adventure', 'Fantasy', 'Romance'","'Love Polygon', 'Video Game', 'Love Polygon', ...",NaN,PG-13 - Teens 13 or older


In [3]:
data.dropna(subset=['Title','Description','Type','Source','Genres','Themes','Demographic','Rating'],inplace=True,axis=0)
data = data.reset_index(drop=True)

In [4]:
data['Description'] = [re.sub(',',' ',t) for t in data['Description']]
data['Genres'] = [re.sub(',',' ',t) for t in data['Genres']]
data['Themes'] = [re.sub(',',' ',t) for t in data['Themes']]
data.head()

,Title,Description,Type,Episodes,Source,Genres,Themes,Demographic,Rating
0,Shingeki no Kyojin,Centuries ago mankind was slaughtered to near...,TV,25,Manga,'Action' 'Award Winning' 'Drama' 'Suspense','Gore' 'Military' 'Survival' 'Gore' 'Milit...,'Shounen',R - 17+ (violence & profanity)
1,Death Note,Brutal murders petty thefts and senseless vi...,TV,37,Manga,'Supernatural' 'Suspense','Psychological','Shounen',R - 17+ (violence & profanity)
2,Fullmetal Alchemist: Brotherhood,After a horrific alchemy experiment goes wrong...,TV,64,Manga,'Action' 'Adventure' 'Drama' 'Fantasy','Military','Shounen',R - 17+ (violence & profanity)
3,One Punch Man,The seemingly unimpressive Saitama has a rathe...,TV,12,Web manga,'Action' 'Comedy','Adult Cast' 'Parody' 'Super Power' 'Adult ...,'Seinen',R - 17+ (violence & profanity)
4,Kimetsu no Yaiba,Ever since the death of his father the burden...,TV,26,Manga,'Action' 'Award Winning' 'Fantasy','Historical','Shounen',R - 17+ (violence & profanity)


We also create a column of combined data that we will use to calculate Cosine Similarity

In [5]:
data["combined"] = data['Title'] + '  ' + data['Description'] + '  ' + data['Type'] + '  ' + data['Source'] + ' ' + data['Genres'] + ' ' + data['Themes'] + ' ' + data['Demographic'] + ' ' + data['Rating']
data.head()

,Title,Description,Type,Episodes,Source,Genres,Themes,Demographic,Rating,combined
0,Shingeki no Kyojin,Centuries ago mankind was slaughtered to near...,TV,25,Manga,'Action' 'Award Winning' 'Drama' 'Suspense','Gore' 'Military' 'Survival' 'Gore' 'Milit...,'Shounen',R - 17+ (violence & profanity),Shingeki no Kyojin Centuries ago mankind was...
1,Death Note,Brutal murders petty thefts and senseless vi...,TV,37,Manga,'Supernatural' 'Suspense','Psychological','Shounen',R - 17+ (violence & profanity),Death Note Brutal murders petty thefts and ...
2,Fullmetal Alchemist: Brotherhood,After a horrific alchemy experiment goes wrong...,TV,64,Manga,'Action' 'Adventure' 'Drama' 'Fantasy','Military','Shounen',R - 17+ (violence & profanity),Fullmetal Alchemist: Brotherhood After a horr...
3,One Punch Man,The seemingly unimpressive Saitama has a rathe...,TV,12,Web manga,'Action' 'Comedy','Adult Cast' 'Parody' 'Super Power' 'Adult ...,'Seinen',R - 17+ (violence & profanity),One Punch Man The seemingly unimpressive Sait...
4,Kimetsu no Yaiba,Ever since the death of his father the burden...,TV,26,Manga,'Action' 'Award Winning' 'Fantasy','Historical','Shounen',R - 17+ (violence & profanity),Kimetsu no Yaiba Ever since the death of his ...


## 2. Calculating cosine similarity and others things

In [6]:
vectorizer = TfidfVectorizer()
matrix = vectorizer.fit_transform(data["combined"])
cosine_similarities = linear_kernel(matrix,matrix)
anime_title = data['Title']
indices = pd.Series(data.index, index=data['Title'])

## 3. Creating a function to recommend anime based on the title

In [7]:
def content_recommender(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:10]
    anime_indices = [i[0] for i in sim_scores]
    return anime_title.iloc[anime_indices]

## 4. Testing

In [12]:
content_recommender("Death Note")

326               Death Note: Rewrite
33                          Kakegurui
26                         Soul Eater
128      Bungou Stray Dogs 3rd Season
138                        Dorohedoro
489                   Ueki no Housoku
491    Makai Ouji: Devils and Realist
309        Yuukoku no Moriarty Part 2
866           Kyuuketsuhime Miyu (TV)
Name: Title, dtype: object

In [8]:
content_recommender("Tokyo Ghoul")

23                Tokyo Ghoul √A
273          Tokyo Ghoul: "Jack"
46                Tokyo Ghoul:re
74     Tokyo Ghoul:re 2nd Season
318         Tokyo Ghoul: "Pinto"
554                      Gantz:O
78                         Akira
123                         Ajin
29                    Elfen Lied
Name: Title, dtype: object

In [9]:
content_recommender("Shingeki no Kyojin")

7                            Shingeki no Kyojin Season 2
9                            Shingeki no Kyojin Season 3
14                    Shingeki no Kyojin Season 3 Part 2
21                  Shingeki no Kyojin: The Final Season
53           Shingeki no Kyojin: The Final Season Part 2
692    Shingeki no Kyojin Season 2 Movie: Kakusei no ...
307    Shingeki no Kyojin: The Final Season - Kankets...
151                               Shingeki no Kyojin OVA
607                        Shingeki no Kyojin: Chronicle
Name: Title, dtype: object